In [ ]:
from pyspark.sql.types import *
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import col, explode, size, array_distinct,collect_list,expr, array_contains, rank,lower,avg, expr, when,mean,split,first,array
import pandas as pd
import matplotlib.pyplot as plt
from pyspark.ml.feature import VectorAssembler, StringIndexer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

spark = SparkSession.builder.getOrCreate()

In [ ]:
profiles = spark.read.parquet('/linkedin/people')


In [ ]:
users = profiles.withColumnRenamed("current_company:name","c_name")
users = users.withColumn("c_name", lower(users["c_name"]))
l = ['yahoo','google','nvidia','microsoft','meta','amazon','apple','intel','oracle','zoom','netflix','spotify','sap','dell','tesla','cisco','linkedin','adobe','qualcomm','amd']

users.filter(users['c_name'] == 'google').select("position").distinct().show(100,truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------+
|position                                                                                                                         |
+---------------------------------------------------------------------------------------------------------------------------------+
|Senior Software Engineer at Google                                                                                               |
|Engineering Manager at Google Search                                                                                             |
|Startup Marketing Manager for Google Cloud                                                                                       |
|Graduate Student at Google                                                                                                       |
|Technical Program Manager                                                  

In [ ]:
#print(users.filter(users['c_name'] == 'intel').count())
FAANG_users = users.filter(users['c_name'].isin(l))
#target_users.display()
print(FAANG_users.count())


14718


In [ ]:
FAANG_users = FAANG_users.withColumn("position_small", lower(col("position")))
words = ["engineer", "cloud", "software", "aiml", "engineering","data", "machine learning","ml","quality assurance","qa","r&d","cyber","security"]
tech_users = FAANG_users.filter(col("position_small").rlike("|".join(words)))

words_to_exclude = ["marketing", "sales", "artist","dei", "helpdesk","assistant","secretary", "null","account","accounts","accounting","area","retired", "human","social","student","manager","art"]
tech_users = tech_users.filter(~col("position_small").rlike("|".join(words_to_exclude)))

positions_list = tech_users.select("position_small").rdd.map(lambda row: row[0]).collect()
#print(len(positions_list))
tech_users.count()
#tech_users =tech_users.select("id","position","url")
#tech_users.sample(fraction=0.0014, seed=42).show(truncate=False)

Out[13]: 4513